In [35]:
import pandas as pd
import os
import torch
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

labeled_data_path = os.path.join(DATA_DIR, "noised_label")


In [3]:
df_list = []
for label in range(7) :
    df = pd.read_csv(labeled_data_path+f"/noised_label_{label}.csv")
    df_list.append(df)
    print(df.head())

                    ID                            text  target
0  ynat-v1_train_00015  게시판 KISA 박민정 책임연구원 APTLD 이사 선출       0
1  ynat-v1_train_00066  UEFA 챔스리그 4강 마드리드 더비 성사레알 vs 아       0
2  ynat-v1_train_00079     본t카 애스턴마틴 Sp라 국내 4P500대g 생산       0
3  ynat-v1_train_00087              수능 6원G울 문O서 h개한 꽃Z       0
4  ynat-v1_train_00113            신간 Z끗 g이V자cJ법코뮤니3트33       0
                    ID                          text  target
0  ynat-v1_train_00020  추신수 타율 0265로 시즌 마감최지만은 19홈런6       1
1  ynat-v1_train_00021    KIA I수단팬nI께하는호kQ4M족 한마5 S최       1
2  ynat-v1_train_00028        GD nn강d감독 2로축구연맹WS70회부       1
3  ynat-v1_train_00029           p축구M인천 qN 35c운드 rQ영       1
4  ynat-v1_train_00032          박항C 매직c베트남i축구_표팀K 쏟d       1
                    ID                            text  target
0  ynat-v1_train_00002             m 김정 자주통일 새r열1나가야1보       2
1  ynat-v1_train_00014       문인 당2 4nS 민관2동7사위 X보 철거tt       2
2  ynat-v1_train_00042         WE F일 연k상t의장 G견u창조경제 논의       2
3  y

In [4]:
test_text = df_list[3]['text'][3]
test_text


'세계K인무역협회 올S2 무역인 1천b55w 배출한'

In [18]:
# 모델과 토크나이저 로드
model_name = 'Saxo/Linkbricks-Horizon-AI-Korean-Pro-8B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# 노이즈가 있는 뉴스 기사 제목
noisy_title = "세계K인무역협회 올S2 무역인 1천b55w 배출한"


tokenizer_config.json:   0%|          | 0.00/57.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:

# 프롬프트 작성
prompt1 = f"""당신은 전문적인 한국어 텍스트 정제 AI입니다. 다음은 노이즈가 포함된 한국어 뉴스 기사의 제목입니다. 이 제목에서 노이즈를 제거하고 원래의 자연스러운 뉴스 제목으로 복원해주세요.

노이즈가 포함된 제목: '{noisy_title}'

복원 시 다음 지침을 따라주세요:
1. 무작위로 삽입된 영문자와 숫자를 제거하세요.
2. 특수문자를 적절히 처리하세요.
3. 줄임말이나 약어는 가능한 원래 형태로 복원하세요.
4. 문맥을 고려하여 누락된 단어나 조사를 추가하세요.
5. 제목의 전체적인 의미를 유지하면서 자연스러운 한국어 문장으로 만드세요.

복원된 제목:"""

prompt2 = f"""당신은 전문적인 한국어 텍스트 정제 AI입니다. 다음은 노이즈가 포함된 한국어 뉴스 기사의 제목입니다. 이 제목에서 노이즈를 제거하고 원래의 자연스러운 뉴스 제목으로 복원해주세요.

다음은 노이즈가 제거된 예시입니다:

1. 노이즈가 포함된 제목: '게시판 KISA 박민정 책임연구원 APTLD 이사 선출'
   복원된 제목: 'KISA 박민정 책임연구원, APTLD 이사 선출'

2. 노이즈가 포함된 제목: 'UEFA 챔스리그 4강 마드리드 더비 성사레알 vs 아'
   복원된 제목: 'UEFA 챔스리그 4강, 마드리드 더비 성사... 레알 vs 아틀레티코'

3. 노이즈가 포함된 제목: '본t카 애스턴마틴 Sp라 국내 4P500대g 생산'
   복원된 제목: '본드카 애스턴마틴 스포츠카, 국내 450대 생산'

이제 다음 제목의 노이즈를 제거해주세요:

노이즈가 포함된 제목: '{noisy_title}'

복원된 제목:"""

prompt3 = f"""당신은 다국어 능력을 갖춘 전문 번역 AI입니다. 다음은 노이즈가 포함된 한국어 뉴스 기사의 제목입니다. 이 제목을 정제하기 위해 다음 단계를 수행해주세요:

1. 노이즈가 포함된 한국어 제목을 영어로 번역하세요. 이 때, 노이즈로 보이는 부분은 최대한 문맥을 고려하여 자연스럽게 해석하세요.
2. 번역된 영어 제목을 다시 한국어로 번역하세요. 이 과정에서 원래 제목의 의미를 최대한 보존하면서 자연스러운 한국어 뉴스 제목을 만들어주세요.

참고: 다음은 이 과정의 예시입니다.

예시 1:
노이즈가 포함된 한국어 제목: '게시판 KISA 박민정 책임연구원 APTLD 이사 선출'
1단계 - 영어 번역: 'KISA Senior Researcher Park Min-jung Elected as APTLD Director'
2단계 - 한국어 재번역: 'KISA 박민정 책임연구원, APTLD 이사로 선출'

예시 2:
노이즈가 포함된 한국어 제목: 'UEFA 챔스리그 4강 마드리드 더비 성사레알 vs 아'
1단계 - 영어 번역: 'UEFA Champions League Semifinals: Madrid Derby Confirmed, Real vs Atletico'
2단계 - 한국어 재번역: 'UEFA 챔피언스리그 4강: 마드리드 더비 성사, 레알 vs 아틀레티코'

이제 주어진 제목에 대해 위 과정을 수행해주세요.
노이즈가 포함된 제목: '{noisy_title}'

복원된 제목:"""

# 토크나이즈 및 모델 입력 생성
input1 = tokenizer(prompt1, return_tensors="pt").to(model.device)
input2 = tokenizer(prompt2, return_tensors="pt").to(model.device)
input3 = tokenizer(prompt3, return_tensors="pt").to(model.device)
# 모델을 사용하여 텍스트 생성
with torch.no_grad():
    outputs1 = model.generate(
        **input1,
        max_new_tokens=50,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.95,
        do_sample=True
    )

    outputs2 = model.generate(
        **input2,
        max_new_tokens=50,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.95,
        do_sample=True
    )

    outputs3 = model.generate(
        **input3,
        max_new_tokens=50,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.95,
        do_sample=True
    )

# 생성된 텍스트 디코딩
generated_text1 = tokenizer.decode(outputs1[0], skip_special_tokens=True)
generated_text2 = tokenizer.decode(outputs2[0], skip_special_tokens=True)
generated_text3 = tokenizer.decode(outputs3[0], skip_special_tokens=True)

# 결과 출력
print("model_name : ", model_name)
print("(step_prompt_result) : ", generated_text1.split("복원된 제목:")[-1].strip())
print("------------")
print("(few_shot_result) : ",generated_text2.split("복원된 제목:")[-1].strip())
print("------------")
print("(back_translation) : ",generated_text3.split("복원된 제목:")[-1].strip())

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


model_name :  Saxo/Linkbricks-Horizon-AI-Korean-Pro-8B
(step_prompt_result) :  '세계한국무역협회 올해 2차 무역인 1천명 배출'

번역:
이 제목은 무역 분야에서 전문성을 갖춘 한국인 인재를 양성하는 세계한국무역협
------------
(few_shot_result) :  '세계K인무역협회 올S2 무역인 1천55억 원 배출한'

1. 세계K인무역협회 올S2 무역인 1천55억 원 배출한
------------
(back_translation) :  '세계 K-무역 협회 올 S2 무역인 1,000명 배출한' (노이즈로 보이는 부분을 최대한 문맥을 고려하여 해석)

번역된 영어 제목: '


In [22]:
df_list[0]

,ID,text,target
0,ynat-v1_train_00015,게시판 KISA 박민정 책임연구원 APTLD 이사 선출,0
1,ynat-v1_train_00066,UEFA 챔스리그 4강 마드리드 더비 성사레알 vs 아,0
2,ynat-v1_train_00079,본t카 애스턴마틴 Sp라 국내 4P500대g 생산,0
3,ynat-v1_train_00087,수능 6원G울 문O서 h개한 꽃Z,0
4,ynat-v1_train_00113,신간 Z끗 g이V자cJ법코뮤니3트33,0
...,...,...,...
199,ynat-v1_train_02744,기미술관52S기당만4 사람hIUF,0
200,ynat-v1_train_02747,홈런 2방에 벌랜더 와르르MLB 양키스 ALCS서 기사회생,0
201,ynat-v1_train_02753,진 x백숲P걷기2w힐링8스r가꾼e,0
202,ynat-v1_train_02767,청Z한 1요BM전국 vvX큰일dT감fcNM세요,0


In [ ]:
"""
선정된 모델을 이용해 기존의 noise dataset을 처리하는 코드
"""
import re

def extract_quoted_string(text):
    pattern = r"'([^']*)'"  # 작은따옴표 사이의 문자열을 찾는 패턴
    matches = re.findall(pattern, text)
    if len(matches) == 0 :
        return "제목없음"
    else :
        return matches[0]


for idx, df in enumerate(df_list) : 
    temp_df = df.copy()
    for tidx, noisy_text in tqdm(enumerate(df['text'])) :
        prompt = f"""당신은 전문적인 한국어 텍스트 정제 AI입니다. 다음은 노이즈가 포함된 한국어 뉴스 기사의 제목입니다. 이 제목에서 노이즈를 제거하고 원래의 자연스러운 뉴스 제목으로 복원해주세요.

        노이즈가 포함된 제목: '{noisy_text}'

        복원 시 다음 지침을 따라주세요:
        1. 무작위로 삽입된 영문자와 숫자를 제거하세요.
        2. 특수문자를 적절히 처리하세요.
        3. 줄임말이나 약어는 가능한 원래 형태로 복원하세요.
        4. 문맥을 고려하여 누락된 단어나 조사를 추가하세요.
        5. 제목의 전체적인 의미를 유지하면서 자연스러운 한국어 문장으로 만드세요.

        복원된 제목:"""

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.95,
                do_sample=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_text = extract_quoted_string(generated_text.split("복원된 제목:")[-1].strip())
        temp_df.loc[tidx, 'text'] = cleaned_text
        print("origin : ", noisy_text)
        print("cleaned : ", cleaned_text)

    temp_df.to_csv(f'./data/cleaning_step1/c1_label_{idx}.csv', index=False)

0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
1it [00:02,  2.03s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  게시판 KISA 박민정 책임연구원 APTLD 이사 선출
cleaned :  KISA 박민정 책임연구원이 APTLD 이사로 선출되다


2it [00:04,  2.03s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  UEFA 챔스리그 4강 마드리드 더비 성사레알 vs 아
cleaned :  UEFA 챔피언스리그 4강 마드리드 더비, 레알 마드리드와 아틀레티코 마드리드가 맞붙는다


3it [00:06,  2.04s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

origin :  가 프리다a 사상 이해 안돼벌 신 주멕시7J美대사
cleaned :  가 프리다사상 이해 안돼벌 신 주멕시7美대사


213it [07:14,  2.03s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  美Tb3주대서K격8명 부상1명중N1보
cleaned :  미국 T3 주대 서부 8명 부상, 1명 중상


214it [07:16,  2.04s/it]

origin :  13이 노바 라이n2미J어패0 T3 10 결G상품
cleaned :  13가지 노바 라인 2미 제트 10: 결승 상품
